In [1]:

import pygame
import sys as sys

pygame 2.6.1 (SDL 2.28.4, Python 3.11.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [ ]:
import pygame, sys

# Initialize Pygame
pygame.init()

# Screen dimensions
WIDTH, HEIGHT = 800, 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Block Collision Game")

# Colors
WHITE = (255, 255, 255)
BLACK = (0, 0, 0)
GRAY = (200, 200, 200)

# Fonts
font = pygame.font.Font(None, 74)
small_font = pygame.font.Font(None, 50)

# Game states
MENU = 0
GAME = 1
OPTIONS = 2
current_state = MENU

# Menu options
menu_options = ["Start Game", "Options", "Exit"]
selected_option = 0

# ---------------------------
# Helper function to load images
def load_images(paths, scale=None):
    images = []
    for path in paths:
        img = pygame.image.load(path).convert_alpha()
        if scale:
            img = pygame.transform.scale(img, scale)
        images.append(img)
    return images

# ---------------------------
# Player classes with directional animations

class Player(pygame.sprite.Sprite):
    def __init__(self, animations, speed, pos):
        """
        animations: a dict with keys "left", "right", "up", "down" and values as lists of images.
        speed: movement speed in pixels per second.
        pos: a pygame.Rect indicating position and size.
        """
        super().__init__()
        self.animations = animations
        self.direction = "down"  # default direction
        self.index = 0
        self.image = self.animations[self.direction][self.index]
        self.rect = pos  # using this for collision detection and drawing
        self.speed = speed
        self.last_update = pygame.time.get_ticks()
        self.animation_delay = 150  # milliseconds between animation frames

    def update_animation(self, moving):
        """Cycle animation if moving; otherwise, reset to the first frame."""
        if moving:
            now = pygame.time.get_ticks()
            if now - self.last_update > self.animation_delay:
                self.last_update = now
                self.index = (self.index + 1) % len(self.animations[self.direction])
                self.image = self.animations[self.direction][self.index]
        else:
            self.index = 0
            self.image = self.animations[self.direction][self.index]

    def move(self, dx, dy):
        """Update the position of the player."""
        self.rect.x += dx
        self.rect.y += dy

class Twin1(Player):
    pass

class Twin2(Player):
    pass

# ---------------------------
# Load animations for each player
# (Adjust file names and scale as needed)
# For Twin1 (green character) animations:
# Movement (walk) animations for Twin1 (green character):
twin1_walk = {
    "left": load_images(["green_brother_left_idle.png", "green_brother_left_walking.png"], scale=(50, 50)),
    "right": load_images(["green_brother_right_idle.png", "green_brother_right_walking.png"], scale=(50, 50)),
    "up": load_images(["green_brother_Rleg_up.png", "green_brother_Lleg_up.png"], scale=(50, 50)),
    "down": load_images(["green_brother_Lleg_down.png", "green_brother_Rleg_down.png"], scale=(50, 50))
}

# Idle animations for Twin1:
twin1_idle = {
    "left": load_images(["green_brother_front_idle.png"], scale=(50, 50))}

# Movement (walk) animations for Twin2 (purple character):
twin2_walk = {
    "left": load_images(["twin2_left_1.png", "twin2_left_2.png"], scale=(50, 50)),
    "right": load_images(["twin2_right_1.png", "twin2_right_2.png"], scale=(50, 50)),
    "up": load_images(["twin2_up_1.png", "twin2_up_2.png"], scale=(50, 50)),
    "down": load_images(["twin2_down_1.png", "twin2_down_2.png"], scale=(50, 50))
}

# Idle animations for Twin2:
twin2_idle = {
    "left": load_images(["twin2_idle_left.png"], scale=(50, 50))}

# Create players using a pygame.Rect for starting position.

player1 = Twin1(twin1_walk, twin1_idle, speed=200, pos=pygame.Rect(100, 100, 50, 50))
player2 = Twin2(twin2_walk, twin2_idle, speed=200, pos=pygame.Rect(400, 300, 50, 50))

# ---------------------------
# Control schemes
controls_player1 = {
    "up": pygame.K_w,
    "down": pygame.K_s,
    "left": pygame.K_a,
    "right": pygame.K_d,
}

controls_player2 = {
    "up": pygame.K_UP,
    "down": pygame.K_DOWN,
    "left": pygame.K_LEFT,
    "right": pygame.K_RIGHT,
}

# ---------------------------
def move_players_with_collision(player1, player2, keys, dt, controls1, controls2):
    # --- Compute movement deltas and update direction ---
    normal_move1 = player1.speed * dt
    dx1 = 0
    dy1 = 0
    if keys[controls1["up"]]:
        dy1 -= normal_move1
        player1.direction = "up"
    if keys[controls1["down"]]:
        dy1 += normal_move1
        player1.direction = "down"
    if keys[controls1["left"]]:
        dx1 -= normal_move1
        player1.direction = "left"
    if keys[controls1["right"]]:
        dx1 += normal_move1
        player1.direction = "right"
    if keys[pygame.K_LSHIFT]:
        dx1 *= 2
        dy1 *= 2

    normal_move2 = player2.speed * dt
    dx2 = 0
    dy2 = 0
    if keys[controls2["up"]]:
        dy2 -= normal_move2
        player2.direction = "up"
    if keys[controls2["down"]]:
        dy2 += normal_move2
        player2.direction = "down"
    if keys[controls2["left"]]:
        dx2 -= normal_move2
        player2.direction = "left"
    if keys[controls2["right"]]:
        dx2 += normal_move2
        player2.direction = "right"
    if keys[pygame.K_LSHIFT]:
        dx2 *= 2
        dy2 *= 2

    # --- Horizontal Movement & Collision ---
    old_x1 = player1.rect.x
    old_x2 = player2.rect.x
    player1.rect.x += dx1
    player2.rect.x += dx2

    if player1.rect.colliderect(player2.rect):
        if abs(dx1) > abs(dx2):
            player1.rect.x = old_x1
        elif abs(dx1) < abs(dx2):
            player2.rect.x = old_x2
        else:
            player1.rect.x = old_x1
            player2.rect.x = old_x2

    # --- Vertical Movement & Collision ---
    old_y1 = player1.rect.y
    old_y2 = player2.rect.y
    player1.rect.y += dy1
    player2.rect.y += dy2

    if player1.rect.colliderect(player2.rect):
        if abs(dy1) > abs(dy2):
            player1.rect.y = old_y1
        elif abs(dy1) < abs(dy2):
            player2.rect.y = old_y2
        else:
            player1.rect.y = old_y1
            player2.rect.y = old_y2

    # --- Update Animations ---
    # 'moving' is True if there is any delta
    player1.update_animation(dx1 != 0 or dy1 != 0)
    player2.update_animation(dx2 != 0 or dy2 != 0)


# ---------------------------
# Drawing functions
def draw_menu():
    screen.fill(WHITE)
    title = font.render("Block Collision Game", True, BLACK)
    screen.blit(title, (WIDTH // 2 - title.get_width() // 2, 100))
    for i, option in enumerate(menu_options):
        color = BLACK if i == selected_option else GRAY
        text = small_font.render(option, True, color)
        screen.blit(text, (WIDTH // 2 - text.get_width() // 2, 250 + i * 70))

def draw_game():
    screen.fill(WHITE)
    # Blit each player's current image.
    screen.blit(player1.image, player1.rect)
    screen.blit(player2.image, player2.rect)

def draw_options():
    screen.fill(WHITE)
    title = font.render("Options", True, BLACK)
    screen.blit(title, (WIDTH // 2 - title.get_width() // 2, 100))
    text = small_font.render("This is the options screen.", True, BLACK)
    screen.blit(text, (WIDTH // 2 - text.get_width() // 2, 300))

# ---------------------------
# Main game loop
clock = pygame.time.Clock()
running = True
while running:
    dt = clock.tick(60) / 1000  # Delta time in seconds

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

        # Handle menu navigation
        if current_state == MENU:
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_DOWN:
                    selected_option = (selected_option + 1) % len(menu_options)
                if event.key == pygame.K_UP:
                    selected_option = (selected_option - 1) % len(menu_options)
                if event.key == pygame.K_RETURN:
                    if selected_option == 0:  # Start Game
                        current_state = GAME
                    elif selected_option == 1:  # Options
                        current_state = OPTIONS
                    elif selected_option == 2:  # Exit
                        running = False

        # Handle options screen
        elif current_state == OPTIONS:
            if event.type == pygame.KEYDOWN:
                if event.key == pygame.K_ESCAPE:  # Go back to menu
                    current_state = MENU

    # ---------------------------
    # Game state handling
    if current_state == GAME:
        keys = pygame.key.get_pressed()
        # Update movement, animation, and collisions together.
        move_players_with_collision(player1, player2, keys, dt, controls_player1, controls_player2)

    # ---------------------------
    # Drawing
    if current_state == MENU:
        draw_menu()
    elif current_state == GAME:
        draw_game()
    elif current_state == OPTIONS:
        draw_options()

    pygame.display.flip()

# Quit Pygame
pygame.quit()
sys.exit()


SystemExit: 

In [1]:
import pygame, sys

# Initialize Pygame
pygame.init()

# Screen dimensions and window setup
WIDTH, HEIGHT = 800, 600
screen = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Green Character Animation Test")
clock = pygame.time.Clock()

# Helper function to load images
def load_images(paths, scale=None):
    images = []
    for path in paths:
        img = pygame.image.load(path).convert_alpha()
        if scale:
            img = pygame.transform.scale(img, scale)
        images.append(img)
    return images

# Movement animations for the green character.
# For each direction, the first image is the idle frame for that direction
# and the second image is the walking frame.
twin1_walk = {
    "left": load_images(["green_brother_left_idle.png", "green_brother_left_walking.png"], scale=(150, 150)),
    "right": load_images(["green_brother_right_idle.png", "green_brother_right_walking.png"], scale=(150, 150)),
    "up": load_images(["green_brother_Rleg_up.png", "green_brother_Lleg_up.png"], scale=(150, 150)),
    "down": load_images(["green_brother_Lleg_down.png", "green_brother_Rleg_down.png"], scale=(150, 150))
}

# Unique idle animations for the green character.
# These images are used when no movement keys are pressed.
twin1_idle = {
    "left": load_images(["green_brother_front_idle.png"], scale=(150, 150)),
    "right": load_images(["green_brother_front_idle.png"], scale=(150, 150)),
    "up": load_images(["green_brother_front_idle.png"], scale=(150, 150)),
    "down": load_images(["green_brother_front_idle.png"], scale=(150, 150))
}

# Player class that uses both movement and idle animations.
class Player(pygame.sprite.Sprite):
    def __init__(self, animations, idle_animations, speed, pos):
        """
        animations: dict with keys "left", "right", "up", "down" for movement animations.
        idle_animations: dict with keys "left", "right", "up", "down" for idle animations.
        speed: Movement speed in pixels per second.
        pos: pygame.Rect for the starting position and size.
        """
        super().__init__()
        self.animations = animations
        self.idle_animations = idle_animations
        self.direction = "down"  # default direction
        self.index = 0
        self.image = self.idle_animations[self.direction][0]
        self.rect = pos
        self.speed = speed
        self.last_update = pygame.time.get_ticks()
        self.animation_delay = 150  # milliseconds delay between animation frames

    def update_animation(self, moving):
        """Cycle through movement frames if moving; otherwise, show idle."""
        if moving:
            now = pygame.time.get_ticks()
            if now - self.last_update > self.animation_delay:
                self.last_update = now
                self.index = (self.index + 1) % len(self.animations[self.direction])
                self.image = self.animations[self.direction][self.index]
        else:
            self.index = 0
            self.image = self.idle_animations[self.direction][0]

    def move(self, dx, dy):
        """Update the player's position."""
        self.rect.x += dx
        self.rect.y += dy

# Function to handle movement and animation update for the player.
def move_and_animate(player, keys, dt):
    dx, dy = 0, 0
    normal_move = player.speed * dt

    # Set direction and compute movement deltas based on WASD keys.
    if keys[pygame.K_w]:
        dy -= normal_move
        player.direction = "up"
    if keys[pygame.K_s]:
        dy += normal_move
        player.direction = "down"
    if keys[pygame.K_a]:
        dx -= normal_move
        player.direction = "left"
    if keys[pygame.K_d]:
        dx += normal_move
        player.direction = "right"
    if keys[pygame.K_LSHIFT]:
        dx *= 2
        dy *= 2

    # Move the player and update the animation.
    player.move(dx, dy)
    player.update_animation(dx != 0 or dy != 0)

# Create a single green character player (centered on screen)
player = Player(twin1_walk, twin1_idle, speed=200, pos=pygame.Rect(375, 275, 50, 50))

# Main game loop
running = True
while running:
    dt = clock.tick(60) / 1000  # Delta time in seconds
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

    keys = pygame.key.get_pressed()
    move_and_animate(player, keys, dt)

    # Clear the screen and draw the current frame.
    screen.fill((255, 255, 255))
    screen.blit(player.image, player.rect)
    pygame.display.flip()

pygame.quit()
sys.exit()


pygame 2.6.1 (SDL 2.28.4, Python 3.11.7)
Hello from the pygame community. https://www.pygame.org/contribute.html


SystemExit: 

c:\Users\simao\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
